# package


In [1]:
# 读取表格
import re
import zipfile
import xml.etree.ElementTree as ET
import xlsxwriter
# 提取数据
import pandas as pd
from pathlib import Path

# 生理数据预处理

## 把组合的excel整合成可以读取的excel

In [6]:
NS_MAIN = "http://schemas.openxmlformats.org/spreadsheetml/2006/main"

def _col_to_idx(cell_ref: str) -> int:
    letters = ''.join(ch for ch in cell_ref if ch.isalpha())
    idx = 0
    for ch in letters:
        idx = idx * 26 + (ord(ch) - 64)
    return idx - 1

def _load_shared_strings(z: zipfile.ZipFile):
    try:
        root = ET.parse(z.open('xl/sharedStrings.xml')).getroot()
    except KeyError:
        return []
    sst = []
    for si in root.findall(f'{{{NS_MAIN}}}si'):
        texts = [t.text or '' for t in si.iter(f'{{{NS_MAIN}}}t')]
        sst.append(''.join(texts))
    return sst

def _list_sheet_entries(z: zipfile.ZipFile):
    wb = ET.parse(z.open('xl/workbook.xml')).getroot()
    rels = ET.parse(z.open('xl/_rels/workbook.xml.rels')).getroot()
    rid2target = {rel.attrib['Id']: rel.attrib['Target'] for rel in rels}
    ns = {'m': NS_MAIN, 'r': "http://schemas.openxmlformats.org/officeDocument/2006/relationships"}
    sheets = wb.find('m:sheets', ns)
    out = []
    for sh in sheets.findall('m:sheet', ns):
        name = sh.attrib['name']
        rid = sh.attrib['{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id']
        target = rid2target[rid]
        path = 'xl/' + target if not target.startswith('xl/') else target
        out.append((name, path))
    return out

def _iter_sheet_rows(z: zipfile.ZipFile, sheet_path: str, sst: list):
    root = ET.parse(z.open(sheet_path)).getroot()
    sheetData = root.find(f'{{{NS_MAIN}}}sheetData')
    if sheetData is None:
        return
    for row in sheetData.findall(f'{{{NS_MAIN}}}row'):
        cells = []
        for c in row.findall(f'{{{NS_MAIN}}}c'):
            r = c.attrib.get('r', 'A1')
            col = _col_to_idx(r)
            while len(cells) <= col:
                cells.append('')
            t = c.attrib.get('t')
            v = c.find(f'{{{NS_MAIN}}}v')
            is_node = c.find(f'{{{NS_MAIN}}}is')

            if t == 's' and v is not None:
                try:
                    idx = int(v.text) if v.text is not None else 0
                    value = sst[idx] if 0 <= idx < len(sst) else ''
                except:
                    value = ''
            elif t == 'inlineStr' and is_node is not None:
                texts = [t_.text or '' for t_ in is_node.iter(f'{{{NS_MAIN}}}t')]
                value = ''.join(texts)
            elif v is not None:
                txt = v.text
                if txt is None:
                    value = ''
                else:
                    try:
                        value = int(txt) if txt.isdigit() or (txt and txt[0] == '-' and txt[1:].isdigit()) else float(txt)
                    except:
                        value = txt
            else:
                value = ''
            cells[col] = value
        yield cells

def export_matching_sheets_pure(src_file: str, dst_file: str, keyword: str, case_sensitive: bool=False):
    with zipfile.ZipFile(src_file) as z:
        sst = _load_shared_strings(z)
        entries = _list_sheet_entries(z)

        def match(name: str):
            return (keyword in name) if case_sensitive else (keyword.lower() in name.lower())

        matched = [(name, path) for name, path in entries if match(name) or 'baseline' in name.lower()]
        if not matched:
            print(f"⚠️ {Path(src_file).name}: 未匹配到包含 “{keyword}” 的 sheet")
            return None

        def sanitize(name):
            s = re.sub(r'[:\\/?*\[\]]', '_', name)[:31]
            return s or "Sheet"

        used = set()
        out_names = []
        for name, _ in matched:
            base = sanitize(name)
            sname, i = base, 1
            while sname in used:
                suf = f"_{i}"
                sname = base[:31-len(suf)] + suf
                i += 1
            used.add(sname)
            out_names.append(sname)

        wb = xlsxwriter.Workbook(dst_file)
        try:
            for (name, path), outn in zip(matched, out_names):
                ws = wb.add_worksheet(outn)
                r = 0
                for row in _iter_sheet_rows(z, path, sst):
                    for c, val in enumerate(row):
                        ws.write(r, c, val)
                    r += 1
        finally:
            wb.close()
        print(f"✅ {Path(src_file).name} 已导出 {len(out_names)} 个 sheet → {Path(dst_file).name}")
        return [n for n, _ in matched], out_names


# === 🚀 批量执行部分 ===
def batch_export_all(input_dir: str, output_dir: str):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for file in sorted(input_path.glob("P*.xlsx")):  # 只处理被试文件
        subject_id = file.stem
        dst_file = output_path / f"{subject_id}_sheets.xlsx"
        export_matching_sheets_pure(str(file), str(dst_file), keyword=subject_id)

    print("\n🎉 全部导出完成！")


# === 用法示例 ===
if __name__ == "__main__":
    input_dir_ECG = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/ECG指标"
    output_dir_ECG = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/ECG指标_读取"
    input_dir_EDA = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/EDA指标"
    output_dir_EDA = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/EDA指标_读取"

    # batch_export_all(input_dir_ECG, output_dir_ECG)
    batch_export_all(input_dir_EDA, output_dir_EDA)

✅ P01.xlsx 已导出 9 个 sheet → P01_sheets.xlsx
✅ P02.xlsx 已导出 9 个 sheet → P02_sheets.xlsx
✅ P03.xlsx 已导出 9 个 sheet → P03_sheets.xlsx
✅ P04.xlsx 已导出 9 个 sheet → P04_sheets.xlsx
✅ P05.xlsx 已导出 9 个 sheet → P05_sheets.xlsx
✅ P06.xlsx 已导出 9 个 sheet → P06_sheets.xlsx
✅ P07.xlsx 已导出 9 个 sheet → P07_sheets.xlsx
✅ P08.xlsx 已导出 10 个 sheet → P08_sheets.xlsx
✅ P10.xlsx 已导出 9 个 sheet → P10_sheets.xlsx
✅ P11.xlsx 已导出 11 个 sheet → P11_sheets.xlsx
✅ P12.xlsx 已导出 9 个 sheet → P12_sheets.xlsx
✅ P13.xlsx 已导出 9 个 sheet → P13_sheets.xlsx
✅ P14.xlsx 已导出 9 个 sheet → P14_sheets.xlsx
✅ P15.xlsx 已导出 9 个 sheet → P15_sheets.xlsx
✅ P16.xlsx 已导出 9 个 sheet → P16_sheets.xlsx
✅ P17.xlsx 已导出 10 个 sheet → P17_sheets.xlsx
✅ P18.xlsx 已导出 9 个 sheet → P18_sheets.xlsx
✅ P20.xlsx 已导出 10 个 sheet → P20_sheets.xlsx
✅ P21.xlsx 已导出 10 个 sheet → P21_sheets.xlsx
✅ P22.xlsx 已导出 9 个 sheet → P22_sheets.xlsx
✅ P23.xlsx 已导出 9 个 sheet → P23_sheets.xlsx
✅ P24.xlsx 已导出 9 个 sheet → P24_sheets.xlsx
✅ P25.xlsx 已导出 9 个 sheet → P25_sheets.xlsx
✅ P27.

## 提取成长表格-ECG

In [4]:
def extract_hrv_modules(excel_path: str, sheet_index: int = 0):
    """
    读取一个 sheet（指定 index），自动识别所有 ECG HRV 模块，
    丢弃非指标列（Recording Name / Subject / Sensor Name / Sensor Description / Segment Count），
    其余全部视为指标，整合为长格式表格。
    """
    # === Step 1. 读取 sheet ===
    xls = pd.ExcelFile(excel_path)
    sheet_name = xls.sheet_names[sheet_index]
    df_raw = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    print(f"🟢 读取 Sheet：{sheet_name}, 大小 {df_raw.shape}")

    # === Step 2. 找出所有 HRV 模块的起始行 ===
    module_rows = df_raw[df_raw[0].astype(str).str.match(r"^ECG HRV", na=False)].index.tolist()
    module_rows.append(len(df_raw))
    print(f"✅ 找到 {len(module_rows)-1} 个 HRV 模块：{module_rows}")

    all_dfs = []

    # === Step 3. 循环提取每个模块 ===
    for i in range(len(module_rows) - 1):
        start, end = module_rows[i], module_rows[i + 1]
        module_name = str(df_raw.iloc[start, 0]).strip()

        header_idx = start + 1
        if header_idx >= len(df_raw):
            print("⚠️ 无字段行，跳过。")
            continue

        # 读取列名与子表
        headers = df_raw.iloc[header_idx].tolist()
        sub_df = df_raw.iloc[header_idx + 1:end].copy()
        sub_df.columns = headers

        # 清理列名：去重 & 去掉 NaN
        sub_df = sub_df.loc[:, ~sub_df.columns.duplicated()]
        sub_df = sub_df.loc[:, sub_df.columns.notna()]

        # === Step 4. 筛选有效 recording ===
        if "Recording Name" not in sub_df.columns:
            print("⚠️ 无 Recording Name 列，跳过。")
            continue
        rec_col = sub_df["Recording Name"].astype(str)
        mask_valid = rec_col.str.lower().str.match(r"p\d{2}-r\d")
        df_valid = sub_df.loc[mask_valid].copy()
        if df_valid.empty:
            print("⚠️ 无有效记录行，跳过。")
            continue

        # === Step 5. 识别指标列 ===
        drop_cols = {"Recording Name", "Subject", "Sensor Name", "Sensor Description", "Segment Count"}
        metric_cols = [c for c in df_valid.columns if c not in drop_cols]
        metric_cols = [c for c in metric_cols if not df_valid[c].isna().all()]
        if not metric_cols:
            print("⚠️ 无可用指标列，跳过。")
            continue

        # === Step 6. 多 Recording 时只保留有数值的那一行 ===
        num_df = df_valid[metric_cols].apply(pd.to_numeric, errors="coerce")
        valid_counts = num_df.notna().sum(axis=1)
        best_idx = valid_counts.idxmax()

        # 若并列，优先有 Mean HR(bpm) 的行
        ties = valid_counts == valid_counts.max()
        if ties.sum() > 1 and "Mean HR(bpm)" in metric_cols:
            has_hr = num_df.loc[ties, "Mean HR(bpm)"].notna()
            if has_hr.any():
                best_idx = has_hr[has_hr].index[0]

        df_pick = df_valid.loc[[best_idx], ["Recording Name"] + metric_cols].copy()

        # === Step 7. 添加元信息 ===
        df_pick["Measure"] = module_name
        df_pick["Subject"] = (
            df_pick["Recording Name"].str.extract(r"(p\d{2})", expand=False)
            .str.replace("p", "", regex=False)
        )

        # 自动识别 Block
        sheet_lower = sheet_name.lower()
        if "baseline" in sheet_lower:
            block_label = "baseline"
        else:
            m = re.search(r"([A-H])", sheet_name)
            block_label = m.group(1) if m else "unknown"
        df_pick["Block"] = block_label

        # === Step 8. 融成长格式 ===
        df_long = df_pick.melt(
            id_vars=["Recording Name", "Subject", "Block", "Measure"],
            value_vars=metric_cols,
            var_name="Metric",
            value_name="Value",
        ).dropna(subset=["Value"], how="any")

        print(f"✅ 模块 {module_name} 提取 {df_long.shape[0]} 行。")
        all_dfs.append(df_long)

    # === Step 9. 汇总所有模块 ===
    if not all_dfs:
        print("⚠️ 未提取到任何有效模块")
        return pd.DataFrame()

    df_all = pd.concat(all_dfs, ignore_index=True)
    print(f"\n✅ 汇总完成：共 {df_all.shape[0]} 行，{df_all['Measure'].nunique()} 个模块")
    return df_all


In [5]:
def batch_extract_hrv(input_dir: str, output_dir: str):
    """
    批量读取一个目录下所有被试 Excel 文件，
    处理每个文件中的所有 sheet，
    并输出为各自的整合 CSV 文件。
    """
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    excel_files = sorted(input_path.glob("P*.xlsx"))
    print(f"🟢 在目录中找到 {len(excel_files)} 个被试文件。")

    for file in excel_files:
        subject_id = file.stem
        print(f"\n============= 处理 {subject_id} =================")

        # 初始化空表
        df_all = pd.DataFrame()

        # 逐个 sheet 读取并整合
        xls = pd.ExcelFile(file)
        for i, sheet in enumerate(xls.sheet_names):
            print(f"  → 读取 sheet: {sheet}")
            df_sub = extract_hrv_modules(file, sheet_index=i)
            if not df_sub.empty:
                df_all = pd.concat([df_all, df_sub], ignore_index=True)

        # 输出 CSV
        if not df_all.empty:
            out_file = output_path / f"{subject_id}_integrated.csv"
            df_all.to_csv(out_file, index=False, encoding='utf-8-sig')
            print(f"✅ 已保存 {out_file.name} ({df_all.shape[0]} 行)")
        else:
            print(f"⚠️ {subject_id} 没有可用数据，跳过导出。")

    print("\n🎉 所有文件处理完成！")


# === 示例用法 ===
if __name__ == "__main__":
    input_dir_ECG2 = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/ECG指标_读取"
    output_dir_ECG2 = "/Volumes/LYY_T7/13_抽象实验数据分析/3_output/ECG_integrated"

    batch_extract_hrv(input_dir_ECG2, output_dir_ECG2)
    

🟢 在目录中找到 35 个被试文件。

============= 处理 P01_sheets =================
  → 读取 sheet: baselineCustom…(1)
🟢 读取 Sheet：baselineCustom…(1), 大小 (60, 14)
✅ 找到 7 个 HRV 模块：[0, 8, 17, 26, 35, 44, 52, 60]
✅ 模块 ECG HRV Time Domain 提取 8 行。
✅ 模块 ECG HRV Frequency Domain ULF Data 提取 3 行。
✅ 模块 ECG HRV Frequency Domain  VLF Data 提取 3 行。
✅ 模块 ECG HRV Frequency Domain  LF Data 提取 4 行。
✅ 模块 ECG HRV Frequency Domain HF Data 提取 4 行。
✅ 模块 ECG HRV Frequency Domain TP & LF/HF Data 提取 2 行。
✅ 模块 ECG HRV Nonlinear 提取 4 行。

✅ 汇总完成：共 28 行，7 个模块
  → 读取 sheet: P01ACustom…(2)
🟢 读取 Sheet：P01ACustom…(2), 大小 (60, 14)
✅ 找到 7 个 HRV 模块：[0, 8, 17, 26, 35, 44, 52, 60]
✅ 模块 ECG HRV Time Domain 提取 8 行。
✅ 模块 ECG HRV Frequency Domain ULF Data 提取 3 行。
✅ 模块 ECG HRV Frequency Domain  VLF Data 提取 3 行。
✅ 模块 ECG HRV Frequency Domain  LF Data 提取 4 行。
✅ 模块 ECG HRV Frequency Domain HF Data 提取 4 行。
✅ 模块 ECG HRV Frequency Domain TP & LF/HF Data 提取 2 行。
✅ 模块 ECG HRV Nonlinear 提取 4 行。

✅ 汇总完成：共 28 行，7 个模块
  → 读取 sheet: P01BCustom…(3)
🟢 读取 Sheet：P0

## 提取成长表格-EDA

In [2]:
def extract_eda_modules(excel_path: str, sheet_index: int = 0):
    """
    从一个 EDA sheet 中提取数据，区分 Time Domain 与 Event Related 模块。
    返回两个 DataFrame：
      - df_time：EDA Time Domain 汇总
      - df_event：EDA Event Related 汇总
    """

    xls = pd.ExcelFile(excel_path)
    sheet_name = xls.sheet_names[sheet_index]
    df_raw = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    print(f"🟢 读取 Sheet：{sheet_name}, 大小 {df_raw.shape}")

    # === 找出所有模块起始行 ===
    module_rows = df_raw[df_raw[0].astype(str).str.match(r"^EDA (Time Domain|Event)", na=False)].index.tolist()
    module_rows.append(len(df_raw))
    # print(f"✅ 找到 {len(module_rows)-1} 个 EDA 模块：{module_rows}")

    all_time_dfs = []
    all_event_dfs = []

    for i in range(len(module_rows) - 1):
        start, end = module_rows[i], module_rows[i + 1]
        module_name = str(df_raw.iloc[start, 0]).strip()
        # print(f"\n🔹 模块 {i+1}: {module_name} ({start} ~ {end})")

        header_idx = start + 1
        if header_idx >= len(df_raw):
            continue

        headers = df_raw.iloc[header_idx].tolist()
        sub_df = df_raw.iloc[header_idx + 1:end].copy()
        sub_df.columns = headers

        # 清理列名
        sub_df = sub_df.loc[:, ~sub_df.columns.duplicated()]
        sub_df = sub_df.loc[:, sub_df.columns.notna()]

        # 识别模块类型
        if re.match(r"^EDA Time Domain", module_name):
            module_type = "time_domain"
        elif re.match(r"^EDA Event Related", module_name):
            module_type = "event_related"
        else:
            continue

        # === 自动识别 Block ===
        sheet_lower = sheet_name.lower()
        if "baseline" in sheet_lower:
            block_label = "baseline"
        else:
            m = re.search(r"([A-H])", sheet_name)
            block_label = m.group(1) if m else "unknown"

        # === Time Domain 模块处理 ===
        if module_type == "time_domain":
            if "Recording Name" not in sub_df.columns:
                continue
            rec_col = sub_df["Recording Name"].astype(str)
            mask_valid = rec_col.str.lower().str.match(r"p\d{2}-r\d")
            df_valid = sub_df.loc[mask_valid].copy()
            if df_valid.empty:
                continue

            drop_cols = {"Recording Name", "Subject", "Sensor Name", "Sensor Description", "Segment Count"}
            metric_cols = [c for c in df_valid.columns if c not in drop_cols and not df_valid[c].isna().all()]
            if not metric_cols:
                continue

            # 多 recording 时取数值最多的一行
            num_df = df_valid[metric_cols].apply(pd.to_numeric, errors="coerce")
            valid_counts = num_df.notna().sum(axis=1)
            best_idx = valid_counts.idxmax()
            df_pick = df_valid.loc[[best_idx], ["Recording Name"] + metric_cols].copy()

            # 添加元信息
            df_pick["Measure"] = module_name
            df_pick["Subject"] = (
                df_pick["Recording Name"].str.extract(r"(p\d{2})", expand=False)
                .str.replace("p", "", regex=False)
            )
            df_pick["Block"] = block_label

            # 融成长表
            df_long = df_pick.melt(
                id_vars=["Recording Name", "Subject", "Block", "Measure"],
                value_vars=metric_cols,
                var_name="Metric",
                value_name="Value",
            ).dropna(subset=["Value"], how="any")

            all_time_dfs.append(df_long)

        # === Event Related 模块处理 ===
        elif module_type == "event_related":
            if "Event Name" not in sub_df.columns:
                continue

            df_valid = sub_df[sub_df["Event Name"].notna()].copy()
            if df_valid.empty:
                continue

            # 删除不需要的列
            drop_cols = {"Event Group", "Recording Name", "Segment Count", "Subject", "Sensor Name", "Sensor Description"}
            df_valid = df_valid.loc[:, [c for c in df_valid.columns if c not in drop_cols]]

            # 添加元信息
            df_valid["Subject"] = (
                df_valid["Event Name"].str.extract(r"(P\d{2})", expand=False)
                .str.lower()
                .str.replace("p", "", regex=False)
            )
            df_valid["Block"] = block_label
            df_valid["Measure"] = module_name

            all_event_dfs.append(df_valid)

    # 汇总
    df_time = pd.concat(all_time_dfs, ignore_index=True) if all_time_dfs else pd.DataFrame()
    df_event = pd.concat(all_event_dfs, ignore_index=True) if all_event_dfs else pd.DataFrame()

    print(f"汇总完成：TimeDomain={df_time.shape}, EventRelated={df_event.shape}")
    return df_time, df_event


In [4]:
def batch_extract_eda(input_dir: str, output_dir: str):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for file in sorted(input_path.glob("P*.xlsx")):
        subject_id = file.stem
        print(f"\n=== 处理 {subject_id} ===")

        df_time_all, df_event_all = pd.DataFrame(), pd.DataFrame()

        xls = pd.ExcelFile(file)
        for i in range(len(xls.sheet_names)):
            df_time, df_event = extract_eda_modules(file, sheet_index=i)
            if not df_time.empty:
                df_time_all = pd.concat([df_time_all, df_time], ignore_index=True)
            if not df_event.empty:
                df_event_all = pd.concat([df_event_all, df_event], ignore_index=True)

        if not df_time_all.empty:
            df_time_all.to_csv(output_path / f"{subject_id}_EDA_TimeDomain.csv", index=False, encoding="utf-8-sig")
        if not df_event_all.empty:
            df_event_all.to_csv(output_path / f"{subject_id}_EDA_EventRelated.csv", index=False, encoding="utf-8-sig")

        print(f"✅ {subject_id} 导出完成。Time={df_time_all.shape}, Event={df_event_all.shape}")

    print("\n🎉 所有文件处理完成！")


In [5]:
# === 示例用法 ===
if __name__ == "__main__":
    input_dir_EDA2 = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/EDA指标_读取"
    output_dir_EDA2 = "/Volumes/LYY_T7/13_抽象实验数据分析/1_rawdata/physio/EDA_integrated"

    # batch_export_all(input_dir_ECG, output_dir_ECG)
    batch_extract_eda(input_dir_EDA2, output_dir_EDA2)


=== 处理 P01_sheets ===
🟢 读取 Sheet：baselineCustom…(1), 大小 (28, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(0, 0)
🟢 读取 Sheet：P01ACustom…(2), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01BCustom…(3), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01HCustom…(4), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01CCustom…(5), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01GCustom…(6), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01DCustom…(7), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01FCustom…(8), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
🟢 读取 Sheet：P01ECustom…(9), 大小 (91, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(57, 10)
✅ P01_sheets 导出完成。Time=(162, 6), Event=(456, 10)

=== 处理 P02_sheets ===
🟢 读取 Sheet：baselineCustom…(1), 大小 (11, 14)
汇总完成：TimeDomain=(18, 6), EventRelated=(0, 0)
🟢 读取 Sheet：P02BCustom…(2), 大小 (68, 14)
汇总